In [3]:
pip install langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [4]:
pip install transformers

In [5]:
pip install torch

In [7]:
pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.5 MB/s eta 0:00:00


In [9]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00


In [11]:
pip install guidance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.4/255.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00


In [4]:
import torch
import os
import numpy as np

from langchain import PromptTemplate
from langchain.chains import LLMChain

from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)

In [2]:
def create_prompt_template(template_str, input_variables):
    """
    Create a PromptTemplate object for use with LangChain.

    Args:
        template_str (str): The string template for the prompt.
        input_variables (list): List of input variable names used in the template.

    Returns:
        PromptTemplate: The LangChain PromptTemplate object.
    """
    return PromptTemplate(
        input_variables=input_variables,
        template=template_str
    )

def create_llm_chain(llm, prompt_template):
    """
    Create an LLMChain object for use with LangChain.

    Args:
        llm (ChatGroq): The language model to use.
        prompt_template (PromptTemplate): The prompt template to use.

    Returns:
        LLMChain: The LangChain LLMChain object.
    """
    return LLMChain(
        llm=llm,
        prompt=prompt_template,
        output_key="response",
        verbose=False
    )

# Define prompt templates
assistant_template = """
You are {sentiment} social media post commenter, you will respond to the following post
Post: "{customer_request}"
Comment:
"""

In [5]:
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)


In [6]:
# Create the LLMChain for generating responses
assistant_chain = LLMChain(
    llm=llm,
    prompt=assistant_prompt_template,
    output_key="assistant_response",
    verbose=False
)

<ipython-input-6-86d759f95554>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  assistant_chain = LLMChain(


In [7]:
# Function to obtain a response to a user comment
def create_dialog(customer_request, sentiment):
    """
    Generate a response to a customer comment based on the provided sentiment.

    Args:
        customer_request (str): The customer's comment or complaint.
        sentiment (str): The sentiment of the response ("nice" or "rude").

    Returns:
        str: The generated response from the assistant.
    """
    # Generate the response using the assistant_chain
    assistant_response = assistant_chain.run(
        {
            "customer_request": customer_request,
            "sentiment": sentiment
        }
    )
    return assistant_response

In [8]:
# Example customer request
customer_request = "Your product is a piece of shit. I want my money back!"

# Generate and print responses with different sentiments
response_nice = create_dialog(customer_request, "nice")
print(f"Nice response: {response_nice}")

response_rude = create_dialog(customer_request, "rude")
print(f"Rude response: {response_rude}")


<ipython-input-7-5c4c85225a4f>:14: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  assistant_response = assistant_chain.run(


Nice response: "Sorry to hear you're not satisfied with our product. That's not the experience we want for our customers. Can you please DM us so we can look into this further and see what we can do to make it right for you?"
Rude response: "Oh great, another genius who can't even be bothered to read the return policy before buying. If you'd spent 5 seconds on our website, you'd know we have a 30-day money back guarantee. Instead, you come here and spew your trash. Try using your brain next time, or better yet, just don't buy anything if you can't handle a little responsibility."
